# Classification

In [ ]:
!pip install -q openai backoff gpt-cost-estimator

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 365.7/365.7 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 24.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 2.6 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import os
from google.colab import drive
drive.mount('/content/drive')
import time
from tqdm.auto import tqdm
from concurrent.futures import ThreadPoolExecutor, as_completed
from gpt_cost_estimator import CostEstimator
import openai
from openai import OpenAI
from google.colab import userdata
import backoff
import random
from sklearn.model_selection import train_test_split

Mounted at /content/drive


In [ ]:
# DataFrame in vier Teile aufteilen

n = len(class_df)
split_size = n // 4

df_parts = [class_df.iloc[i*split_size:(i+1)*split_size] for i in range(4)]

save_path = '/content/drive/MyDrive/Projekt_Mobilisierung/Data/class'
os.makedirs(save_path, exist_ok=True)

for idx, part in enumerate(df_parts):
    file_name = f'part_{idx+1}.csv'
    file_path = os.path.join(save_path, file_name)
    part.to_csv(file_path, index=False)

print("DataFrames wurden erfolgreich in vier Teile aufgeteilt und gespeichert.")

DataFrames wurden erfolgreich in vier Teile aufgeteilt und gespeichert.


In [ ]:
class_df = pd.read_csv('/content/drive/MyDrive/Projekt_Mobilisierung/Data/class/part_4.csv')
example_df = pd.read_csv('/content/drive/MyDrive/Projekt_Mobilisierung/Data/example_df_5.csv')

In [ ]:
class_df = class_df.rename(columns={'body': 'posttext'})
class_df

,id,posttext,author,name,timestamp,type,url,hashtags,likes,comments,...,Selbstpräsentation politisch,Parteipräsentation unpolitisch,Parteipräsentation politisch,Interaktion,Neutrale Infos zur Stimmabgabe,Mitgliederinfo,Fundraising,Veranstaltungshinweis,Wahlaufruf direkt,Sonstiges
0,CwXDd_BN0uk,NaN,harryscheuenstuhl,Harry Scheuenstuhl,2023-08-25 08:05:14,photo,https://www.instagram.com/p/CwXDd_BN0uk,NaN,17,0,...,0,0,0,0,0,0,0,0,0,0
1,CwXDYe5N5yx,NaN,harryscheuenstuhl,Harry Scheuenstuhl,2023-08-25 08:04:29,photo,https://www.instagram.com/p/CwXDYe5N5yx,NaN,15,0,...,0,0,0,0,0,0,0,0,0,0
2,CwUKoLRISWf,NaN,harryscheuenstuhl,Harry Scheuenstuhl,2023-08-24 05:10:04,photo,https://www.instagram.com/p/CwUKoLRISWf,NaN,17,0,...,0,0,0,0,0,0,0,0,0,0
3,CwUJ_6FNdKC,Holzmarkt Kerwa in Bad Windsheim mit Braumeist...,harryscheuenstuhl,Harry Scheuenstuhl,2023-08-24 05:04:34,photo,https://www.instagram.com/p/CwUJ_6FNdKC,NaN,11,0,...,0,0,0,0,0,0,0,0,0,0
4,CwSF5jENzOr,"Zwischen Ernte Gurken, Tomaten, Bohnen",harryscheuenstuhl,Harry Scheuenstuhl,2023-08-23 09:50:16,photo,https://www.instagram.com/p/CwSF5jENzOr,NaN,6,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11719,CtEak1vIzCH,NaN,zwingmannmichael,Michael Zwingmann,2023-06-04 12:47:44,video,https://www.instagram.com/p/CtEak1vIzCH,NaN,11,0,...,0,0,0,0,0,0,0,0,0,0
11720,CtCaknTt7yK,Herzlichen Glückwunsch 🎉,zwingmannmichael,Michael Zwingmann,2023-06-03 18:10:00,video,https://www.instagram.com/p/CtCaknTt7yK,NaN,24,0,...,0,0,0,0,0,0,0,0,0,0
11721,CtAEpdvrORy,Gerne bin ich als 2. Bürgermeister der Einladu...,zwingmannmichael,Michael Zwingmann,2023-06-02 20:18:49,photo,https://www.instagram.com/p/CtAEpdvrORy,NaN,31,1,...,0,0,0,0,0,0,0,0,0,0
11722,Csv3bz3qWar,Ich wünsche euch allen ein schönes Pfingstwoch...,zwingmannmichael,Michael Zwingmann,2023-05-27 13:15:31,photo,https://www.instagram.com/p/Csv3bz3qWar,NaN,28,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
api_key_name = "xxx"
api_key = "xxx"

client = OpenAI(
    api_key=api_key
)

@CostEstimator()
def query_openai(model, messages, mock=True, completion_tokens=10):
    return client.chat.completions.create(
                      model=model,
                      messages=messages,
                      max_tokens=600)

@backoff.on_exception(backoff.expo, (openai.RateLimitError, openai.APIError))
def run_request(system_prompt, user_prompt, model, mock):
  messages = [
    {"role": "system", "content": system_prompt},
    {"role": "user", "content": user_prompt}
  ]

  return query_openai(
          model=model,
          messages=messages,
          mock=mock
        )

In [ ]:
category_names = [
    "Politischer Inhalt/Info",
    "Negative Campaigning",
    "Selbstpräsentation unpolitisch",
    "Selbstpräsentation politisch",
    "Parteipräsentation unpolitisch",
    "Parteipräsentation politisch",
    "Interaktion",
    "Neutrale Infos zur Stimmabgabe",
    "Mitgliederinfo",
    "Fundraising",
    "Veranstaltungshinweis",
    "Wahlaufruf direkt",
    "Sonstiges"
]

def extract_categories(row):
    categories = [category_names[i] for i, cat in enumerate(row[6:19]) if cat == 1]
    return ", ".join(categories)

example_text = "\n\n".join(
    f"posttext: {row['posttext']}\nText im Bild: {row['ocr_text']}\nHashtags: {row['hashtags']}\nKategorien: {extract_categories(row)}"
    for index, row in example_df.iterrows()
)

In [ ]:
len(example_text)

133429

In [ ]:
system_prompt = """
Du bist ein fortgeschrittenes KI-Modell zur Klassifikation. Deine Aufgabe ist es, Instagram-Posts von Kandidierenden aus dem vergangenen Landtagswahlkampf 2023 in Bayern nach den Kommunikationsstrategien zu klassifizieren. Ziel der Untersuchung ist es, Handlungsmuster in Wahlkampfposts zu quantifizieren und zu verstehen, welche Strategien mit Blick auf die Performance und Reichweite der Posts am besten funktionieren. Die Kodierungen helfen dabei zu verstehen, wie Wahlkampf in sozialen Medien verbessert werden kann und was das für demokratische Parteien bedeutet.

Bei der Klassifikation sollen die Beiträge in zwölf Kategorien klassifiziert werden. Mehrfachnennungen sind möglich, ein Post kann mehrere Handlungsmuster abbilden. Der theoretische Unterbau meiner Arbeit geht davon aus, dass Politiker:innen folgende zwölf Motive in sozialen Medien verfolgen:

1. **Politischer Inhalt/Info**: Politische Inhalte und Themen werden bearbeitet, oft verknüpft mit Forderungen oder wie man selbst oder die eigene Partei dazu steht. Weit gefasst, auch die Forderung nach einem „gerechterem Bayern“ reicht hier schon aus.
   - Beispiel: "Wir setzen uns für ein gerechteres Bayern ein, indem wir..."

2. **Negative Campaigning**: Die politische Konkurrenz wird diskreditiert und/oder vor allem kritisiert. Meist inhaltlich geprägt. Eher streng, es muss der Gegner namentlich genannt werden. Es müssen also andere Parteien (CSU, SPD, FDP, AFD, Grüne, Freie Wähler, Linke) oder die Staatsregierung in Bayern, die Bundesregierung (Ampel) in Deutschland oder die Opposition genannt werden.
   - Beispiel: "Die CSU hat in den letzten Jahren versagt, weil..."

3. **Selbstpräsentation unpolitisch**: Die eigene Person der Kandidierenden wird in unpolitischem Kontext präsentiert. Oft geht es darum, Nähe herzustellen und sich als „normale“ Person zu präsentieren. Oft werden positive Themen wie Ferien, Urlaub, Feiern oder schönes Wetter bedient.
   - Beispiel: "Ich genieße die Zeit mit meiner Familie im Urlaub."

4. **Selbstpräsentation politisch**: Die eigene Person wird in politischem Kontext präsentiert. Entweder die eigene politische Arbeit in Mandat, Partei, oder der Besuch von Institutionen im politischen Kontext. Auch die Präsentation der eigenen analogen Wahlkampfmethoden (Plakate, Infostand, Haustürbesuche, Podiumsdiskussionen) gehört hier dazu. Auch die Präsentation der eigenen Arbeit im vorpolitischen Raum (Vereine und Organisationen) gehört hier dazu. Wichtig ist hier, dass die Kategorie immer mit einem tatsächlichen Ereignis in der realen Welt verbunden ist, von der die Kandidierenden berichten und auch selbst dabei waren. Also der Besuch einer Firma vor Ort oder der Bericht von Haustürbesuchen. Die Präsentation der eigenen politischen Standpunkte fällt hier nicht mit hinein, das ist Kategorie 1.
   - Beispiel: "Heute habe ich die Firma XY besucht und mich über ihre Innovationen informiert."

5. **Parteipräsentation unpolitisch**: Die eigene Partei wird in unpolitischem Kontext präsentiert. Oft geht es darum, Nähe herzustellen und sich als normale Person zu präsentieren. Oft werden positive Themen wie Ferien, Urlaub, Feiern oder schönes Wetter bedient. Im Vergleich zu Kategorie 3 geht es hier um den Partei-Kontext, also ob die Themen und Veranstaltungen gemeinsam besucht/präsentiert werden. Typischerweise wird dann davon berichtet, dass Veranstaltungen gemeinsam besucht wurden (Stichwort „wir“) und es werden anderen Politiker:innen oder die Partei selbst und ihre Untergliederungen markiert mit dem @-Tag. Wenn Kategorie 5 gewählt wird, schließt das Kategorie 3 nicht aus, die Person des Kandidierenden wird dann auch immer mit präsentiert.
   - Beispiel: "Gemeinsam mit meinen Parteikolleg:innen genieße ich das schöne Wetter."

6. **Parteipräsentation politisch**: Die eigene Partei wird in politischem Kontext präsentiert. Entweder die eigene politische Arbeit in Mandat, Partei, oder der Besuch von Institutionen im politischen Kontext. Typisch ist auch der Besuch eines „Partei-VIPs“. Im Vergleich zu Kategorie 3 geht es hier um den Partei-Kontext, also ob die Themen und Veranstaltungen gemeinsam besucht/präsentiert werden. Zusätzlich zählt hierzu die Präsentation von Umfrageergebnissen. Typischerweise wird dann davon berichtet, dass Veranstaltungen gemeinsam besucht wurden (Stichwort „wir“) und es werden anderen Politiker:innen oder die Partei selbst und ihre Untergliederungen markiert mit dem @-Tag. Wenn Kategorie 6 gewählt wird, schließt das Kategorie 4 nicht aus, die Person des Kandidierenden wird dann auch immer mit präsentiert, da er zeigt, dass er in der Partei gut vernetzt ist und dort gute Arbeit leistet. Die Präsentation der parteipolitischen Standpunkte fällt hier nicht mit hinein, das ist Kategorie 1.
   - Beispiel: "Mit dem Parteivorsitzenden haben wir heute die neue Kampagne gestartet."

7. **Interaktion**: Instagram wird von den Kandidierenden genutzt, um mit dem Wahlvolk in Kontakt zu treten. Typisch sind Aufforderungen in die Kommentare zu schreiben oder sich per Direktnachricht zu melden. Auch Fragezeichen und Fragen zum Post bzw. zur Meinung der Menschen sind deutliche Hinweise für diese Kategorie. Die Interaktion muss aber immer im digitalen Raum stattfinden, die Aufforderung zum Wahlkampfstand vorbei zukommen zählt hier nicht.
   - Beispiel: "Was haltet ihr von unserem neuen Programm? Kommentiert unten!"

8. **Neutrale Infos zur Stimmabgabe**: Es wird neutral über den Ablauf der Stimmabgabe informiert, ohne für die eigene Person oder Partei zu werben. Ein Hinweis auf welcher Liste man selbst zu finden ist reicht hier nicht aus, es muss ein längerer detaillierter Text sein, der darauf hinweist, wie die Wahlabgabe im Detail funktioniert.
   - Beispiel: "So funktioniert die Briefwahl: Schritt 1..."

9. **Mitgliederinfo**: Es wird spezifisch mit den Mitgliedern der eigenen Partei/Organisation kommuniziert, um interne Themen zu bearbeiten.
   - Beispiel: "Liebe Mitglieder, bitte denkt an unsere nächste Versammlung am Freitag."

10. **Fundraising**: Es wird versucht, finanzielle Mittel für Wahlkampf und generell für die politische Arbeit einzutreiben, meistens als Spendenaufruf.
    - Beispiel: "Unterstützt unseren Wahlkampf mit einer Spende!"

11. **Veranstaltungshinweis**: Es wird auf eine (Wahlkampf)-Veranstaltung hingewiesen oder eingeladen. Die Veranstaltung muss in der Zukunft liegen, ein Bericht über eine vergangene Veranstaltung zählt hier nicht, das wäre Kategorie 4 und/oder 6.
    - Beispiel: "Kommt zu unserer Wahlkampfveranstaltung am Samstag!"

12. **Wahlaufruf direkt**: Es wird offen dazu aufgerufen, dass die eigene Partei oder Person gewählt werden soll. Oft verbunden mit Infos zum Ablauf der Wahl. Die reine Nennung auf welchem Listenplatz man steht reicht hier nicht aus.
    - Beispiel: "Wählt mich am 15. Oktober für ein besseres Bayern!"

13. **Sonstiges/KA**: Nur wenn keine der 12 Kategorien zutrifft, kann nur exklusiv ausgewählt werden. Es geht also nicht, dass 12 und eine andere Kategorie klassifiziert werden.

Hier sind einige klassifizierte Beispiele:
{example_text}

Bei der Klassifikation bekommst du den Text des Beitrags, eine Bildbeschreibung, die Schrift im Bild und die Hashtags zur Verfügung gestellt. Deine Aufgabe ist es zu entscheiden, ob die Strategien 1-12 im Post vorliegen. Gib die zutreffenden Kategorien als Liste zurück. Wenn keine der Kategorien zutrifft, gib "Sonstiges/KA" zurück, dass kommt allerdings kaum vor.

"""

In [ ]:
prompt = """
Bitte klassifiziere die Posts aus dem Datensatz anhand der Spalten „posttext“, „ocr_text“, "caption" und „hashtags“. Du kannst mehrere Kategorien als zutreffend klassifizieren (Multi-shot). Hier die Kategorien:
„1. Politischer Inhalt/Info“
„2. Negative Campaigning“
„3. Selbstpräsentation unpolitisch“
„4. Selbstpräsentation politisch“
„5. Parteipräsentation unpolitisch“
„6. Parteipräsentation politisch“
„7. Interaktion“
„8. Neutrale Infos zur Stimmabgabe“
„9. Mitgliederinfo“
„10. Fundraising“
„11. Veranstaltungshinweis“
„12. Wahlaufruf direkt“
„13. Sonstiges“
"Posttext: [posttext]\n"
"ocr_text: [ocr_text]\n"
"Caption: [caption]\n"
"Hashtags: [hashtags]"
"""

In [ ]:
CostEstimator.PRICES['gpt-4o-mini'] = {
    'input': 0.15/1000,
    'output': 0.60/1000
}

MOCK = False
RESET_COST = True
COLUMN = 'Kategorie'
SAMPLE_SIZE = 0
MODEL = "gpt-4o-mini"

if COLUMN not in class_df.columns:
    class_df[COLUMN] = None

categories = [
    "Politischer Inhalt/Info",
    "Negative Campaigning",
    "Selbstpräsentation unpolitisch",
    "Selbstpräsentation politisch",
    "Parteipräsentation unpolitisch",
    "Parteipräsentation politisch",
    "Interaktion",
    "Neutrale Infos zur Stimmabgabe",
    "Mitgliederinfo",
    "Fundraising",
    "Veranstaltungshinweis",
    "Wahlaufruf direkt",
    "Sonstiges"
]

for category in categories:
    if category not in class_df.columns:
        class_df[category] = 0

if RESET_COST:
    CostEstimator.reset()
    print("Reset Cost Estimation")

filtered_df = class_df.copy()

if not filtered_df.empty:
    if SAMPLE_SIZE > 0:
        SAMPLE_SIZE = min(SAMPLE_SIZE, len(filtered_df))
        filtered_df = filtered_df.sample(SAMPLE_SIZE)

    for index, row in tqdm(filtered_df.iterrows(), total=len(filtered_df)):
        try:
            combined_text = (
                f"posttext: {str(row['posttext'])}\n"
                f"OCR_Text: {str(row['ocr_text'])}\n"
                f"Caption: {str(row['caption'])}\n"
                f"Hashtags: {str(row['hashtags'])}"
            )

            p = prompt.replace('[posttext]', str(row['posttext']))
            p = p.replace('[ocr_text]', str(row['ocr_text']))
            p = p.replace('[caption]', str(row['caption']))
            p = p.replace('[hashtags]', str(row['hashtags']))

            response = run_request(system_prompt, p, MODEL, MOCK)

            if not MOCK:
                response_content = response.choices[0].message.content
                for category in categories:
                    class_df.at[index, category] = 0

                for category in categories:
                    if category.lower() in response_content.lower():
                        class_df.at[index, category] = 1

        except Exception as e:
            import traceback
            print(f"An error occurred: {e}")
            traceback.print_exc()

else:
    print("No rows to process.")

print()

file_path = "/content/drive/MyDrive/Projekt_Mobilisierung/Data/class/class_4_final_df_pm.csv"
class_df.to_csv(file_path, index=False)

Reset Cost Estimation


  0%|          | 0/11724 [00:00<?, ?it/s]

Die letzten 5000 Zeilen der Streamingausgabe wurden abgeschnitten.
Cost: $0.0001 | Total: $1.6967


Quellen:

Achmann-Denkler, M. (2024). michaelachmann/social-media-lab: DOI Release (v0.0.12). Zenodo. https://doi.org/10.5281/zenodo.10618621

Achmann-Denkler, M. (2024). “Text Classification.” January 22, 2024. https://doi.org/10.5281/zenodo.10039756.